In [1]:
text = open('book.txt','r',encoding='utf8').read()
text = text.lower()

In [2]:
# split whole text in list of sentences
sentences = text.split('\n')

In [3]:
import tensorflow as tf
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [4]:
# now we are going to tokenize our text data i.e sentences which mean we are going to assign every word a unique integer
# so to do that first we have to declare object of tokenizer

# their are several argument we cann pass in tokenizer but we only make oov_token = <UNK> (unknown)
# we use this as may be their are several word in data which are not present in our vocbulary(dictionary words from text)
# so if we didn't pass it skip those word as its default value is NONE so we initialize it as unknown

tokenizer = Tokenizer(oov_token='<UNK>')



In [5]:
tokenizer.fit_on_texts(sentences)


In [6]:
vocab_size = len(tokenizer.word_index)+1
vocab_size

8933

In [7]:
# now we will convert our sentences into integr sequences i.e for every sentence it going to replace word in sentence with corresponding 
# integer value that we get through fit_on_text function

sequences = tokenizer.texts_to_sequences(sentences)


In [8]:
# now we convert our sequences into n-gram sequences
# for ex we have sequence as [1,3,21,5,6]
# so with this we can make more sequences as [1,3],[1,3,21],[1,3,21,5],[1,3,21,5,6] (we keep min two word as one sequences)
input_sequences = []
for sq in sequences:
  for i in range(1,len(sq)):
    n_gram_sequence = sq[:i+1]
    input_sequences.append(n_gram_sequence)

# lets print first two sequences
print(input_sequences[0])
print(input_sequences[1])




[146, 4790]
[146, 4790, 2]


In [9]:
# now we are going to pad our sequence because a we are going to use RNN in our project and it require input of same length
# so what padding does it make make all sequence of same length by adding required no. of 0 in front of orginal seuence

# for example we have [124,470] and in our data we have max length of sequence is 5 so it will convert it into [0,0,0,146,4790]

# so first we find maxlength
max_length = max([len(sq) for sq in input_sequences])
max_length

20

In [10]:
padded_sequences = pad_sequences(input_sequences,max_length)
print(padded_sequences[0])
print(padded_sequences[1])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0  146 4790]
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0  146 4790    2]


In [11]:
# lets covert or padded sequences into numpy arry for easy computation
import numpy as np
padded_sequences = np.array(padded_sequences)

In [12]:
# now we are going to make our input output pair or data 
# so for that what we do as we have have sequences of len 20 we kept 19 as input data and the last one as output data
# basic idea is we have some text sentence and we can predict what is the next word

x= padded_sequences[:,:-1] # i.e all row and column(expect last one)
labels=padded_sequences[:,-1]

In [13]:
# since in this we are going to predict next word from some given word so basicall it is a kind of classification problem
# so we convert our label data into one hot encoded format as it is the requirment of neural networks

y=tf.keras.utils.to_categorical(labels, num_classes=vocab_size)

In [14]:
x.shape

(101619, 19)

In [15]:
y.shape

(101619, 8933)

In [17]:
# import necessary library
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,Bidirectional,LSTM,Dense
from tensorflow.keras.optimizers import Adam

In [20]:
# define and compile the model
model = Sequential()
model.add(Embedding(vocab_size,100,input_length=max_length-1))
model.add(Bidirectional(LSTM(256)))
model.add(Dense(vocab_size,activation='softmax'))
adam = Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy',optimizer=adam,metrics = ['acc'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 19, 100)           893300    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 512)               731136    
_________________________________________________________________
dense_1 (Dense)              (None, 8933)              4582629   
Total params: 6,207,065
Trainable params: 6,207,065
Non-trainable params: 0
_________________________________________________________________


In [21]:
# initialize the callback function to early stop the training of data if their is not atleast 1% improvment in accuracy
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor ='acc',min_delta=0.01) 

In [22]:
model.fit(x,y,epochs=50,verbose=1,batch_size=512,callbacks=[es])

Epoch 1/50
199/199 [==============================] - 40s 42ms/step - loss: 6.6591 - acc: 0.0605
Epoch 2/50
199/199 [==============================] - 8s 41ms/step - loss: 5.2990 - acc: 0.1409
Epoch 3/50
199/199 [==============================] - 8s 41ms/step - loss: 4.6660 - acc: 0.1766
Epoch 4/50
199/199 [==============================] - 8s 42ms/step - loss: 4.0851 - acc: 0.2122
Epoch 5/50
199/199 [==============================] - 8s 42ms/step - loss: 3.5529 - acc: 0.2658
Epoch 6/50
199/199 [==============================] - 8s 42ms/step - loss: 3.1028 - acc: 0.3242
Epoch 7/50
199/199 [==============================] - 8s 42ms/step - loss: 2.7155 - acc: 0.3855
Epoch 8/50
199/199 [==============================] - 8s 42ms/step - loss: 2.4045 - acc: 0.4387
Epoch 9/50
199/199 [==============================] - 8s 42ms/step - loss: 2.2014 - acc: 0.4779
Epoch 10/50
199/199 [==============================] - 8s 42ms/step - loss: 1.9904 - acc: 0.5218
Epoch 11/50
199/199 [=================

In [27]:
# time for story telling or prediction
seed_text = " I warned you about this thing and"
next_words = 50
for _ in range(next_words):
  sequence = tokenizer.texts_to_sequences([seed_text])
  padded = pad_sequences(sequence,maxlen=max_length-1)
  predicted = np.argmax(model.predict(padded,verbose=0),axis=1)
  output_word =''
  for word,index in tokenizer.word_index.items():
    if index==predicted:
      output_word = word
      break
  seed_text += ' ' + output_word
print(seed_text)


 I warned you about this thing and i have been waiting in the same state of affairs rather roughly the man who had written the note itself handed so far as it bore a little you or have the worst of all the proofs our little to him ” i remarked “that homely as usual as she
